In [1]:
#TAREA 2 DIEGO POBLETE ALBERTO YEH

In [2]:
from finrisk import QC_Financial_3 as Qcf
from scipy.stats import norm # Para poder hacer el normcdf del d1
from typing import List, Tuple, Callable
from scipy.interpolate import interp1d

import sys
sys.path.insert(1, '../modules')
import auxiliary as aux
# from modules
import auxiliary as aux

from numpy import random as rnd
import plotly.express as px
import pandas as pd
import numpy as np
import math

In [3]:
frmt = {'tasa': '{:.4%}', 'df': '{:.6%}'} 

#se obtienen los valores de la curva cero cupón en la data

curva = pd.read_excel('../data/20201012_built_sofr_zero.xlsx')

curva['t'] = curva['plazo'] / 365.0

curva['rate'] = np.log(1 / curva['df'])/( curva['plazo'] / 365.0)

#interpolando la curva 
zcurva = interp1d(curva['t'],
                  curva['rate'],
                  kind='cubic',
                  fill_value="extrapolate")

In [4]:
#generando variables auxiliares con diferencias finitas
def zrate(t: float) -> float:
    return zcurva(t)


def dzrate(t: float) -> float:
    delta = .0001
    return (zcurva(t + delta) - zrate(t)) / delta


def d2zrate(t: float) -> float:
    delta = .0001
    return (dzrate(t + delta) - dzrate(t)) / delta

#definir las funciones fwd(0,t) y dfwd(0,t).
def fwd(t: float) -> float:
    return zrate(t) + t * dzrate(t)

def dfwd(t: float) -> float:
    return 2 * dzrate(t) + t * d2zrate(t)

In [5]:
# Parametros
strike = 1
gamma = 0.5
sigma = 0.015
num_sim = 1000
num_steps = 254
seed = 2556
# Proxy de la tasa instantánea r(t) es una tasa entre t y t + dt (donde dt es un intervalo infitesimal)
r0 = zrate(0)
t = 1
T_1 = 1
T_2 = 2

In [6]:
#obteniendo theta
def theta(t: float) -> float:
    aux = (sigma ** 2) / (2.0 * gamma) * (1 - math.exp(-2.0 * gamma * t))
    return dfwd(t) + gamma * fwd(t) + aux

In [7]:
#matrices auxiliares para valorizar un bono cero cupón 

#obteniendo función B
def b_hw(gamma: float, t: float, T: float) -> float:
    """
    Calcula el valor de la función B(t,T) que interviene en la fórmula
    para el valor de un bono cupón cero en el modelo de HW.
    
    params:
    
    - gamma: intensidad de reversión del modelo HW
    - t:
    - T:
    
    return:
    
    - valor de la función B(t, T)
    """
    aux = 1 - math.exp(- gamma * (T - t))
    return aux / gamma

#obteniendo función A
def a_hw(zrate: float, fwd, gamma: float, sigma: float, t: float, T: float,
         verbose = False):
    """
    verbose: cuando es True imprime los valores de c1, c2 y c3.
    Calcula el A(t,T) dentro de la formula de Z para sacar el valor del bono cupon cero
    """
    b = b_hw(gamma, t, T)
    dfT = math.exp(-zrate(T) * T) # en plazo final
    dft = math.exp(-zrate(t) * t) # en plazo inicial
    c1 = math.log(dfT / dft)
    c2 = b * fwd(t)
    c3 = (sigma**2) / (4 * gamma) * (b**2) * (1 - math.exp(-2 * gamma * t))
    if verbose:
        print("c1: " + str(c1))
        print("c2: " + str(c2))
        print("c3: " + str(c3))
    return c1 + c2 - c3

In [8]:
#obteniendo finalmente la valorización de un bono cupón a 1Y
def zero_hw(r: float, gamma: float, sigma: float, 
            zrate: float, fwd: float, t: float, T: float) -> float:
    """
    Retorna el valor de Z(r,t,T) de la formula tomando las funciones a_hw y b_hw asi obteniendo
    el valor del bono cupon cero
    Modelo de HW reproduce exactamente los valores de los factores de descuento de la curva 
    cupon cero de mercado
    
    """
    a = a_hw(zrate, fwd, gamma, sigma, t, T)
    b = b_hw(gamma, t, T)
    return math.exp(a - b * r)



In [9]:
#simulando la dinámica de la tasa con 264 saltos temporales 
def sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed = None):
    """
    """
    dt = 1 / 264.0
    num_steps += 1
    
    # Calcula los números aleatorios
    alea = np.zeros((num_sim, num_steps))
    rnd.seed(seed)

    for i in range(0, num_sim):
        for j in range(0, num_steps):
            alea[i][j] = rnd.normal()
            
    # Calcula los valores de Theta. Theta sólo depende del tiempo, no de la simulación. 
    theta_array = np.zeros(num_steps)
    tiempo = np.zeros(num_steps)
    for i in range(0, num_steps):
        tiempo[i] = i * dt
        theta_array[i] = theta(i * dt)
    
    # Simula las trayectorias
    sqdt_sigma = math.sqrt(dt) * sigma
    gamma_dt = gamma * dt
    sim = np.zeros((num_sim, num_steps))
    for i in range(0, num_sim):
        sim[i][0] = r0
        r = r0
        for j in range(1, num_steps):
            r = r + theta_array[j - 1] * dt - gamma_dt * r + sqdt_sigma * alea[i][j - 1]
            sim[i][j] = r
    return tiempo, sim

In [10]:
tiempo, sim = sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed )

In [11]:
#se obtiene la multiplicación de los factores de dcto simulados desde el día 1 hasta el día 263
#esto es para obtener el valor presente del instrumento 

dt = 1/264

tiempo, s = sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed)
df = []
D = s[:len(s)-1]
for sim in D:
    aux_df = math.exp(-dt * np.sum(sim))
    df.append(aux_df)
    

In [12]:
############ PREGUNTA 1: VALORIZANDO CALL Y PUT ############

In [13]:
#se obtiene el valor de la opción call y put con la formula empírica en el instante actual 
def Bono_call_put(gamma, T_1, T_2, sigma, r, zrate ,fwd ,t, strike ):

    auxB = b_hw(gamma, T_1, T_2)
    auxright = math.sqrt((sigma**2/(2*gamma)) * (1 - math.exp(-2 * gamma * T_1)))
    Sz = auxB * auxright
    
    Zt2 = zero_hw(r, gamma, sigma, zrate, fwd, t, T_2)
    Zt1 = zero_hw(r, gamma, sigma, zrate, fwd, t, T_1)
    
    auxlog = math.log(Zt2/(strike*Zt1))
    auxSz = 0.5 * Sz**2
    d1 = (auxlog + auxSz)/Sz    
    d2 = d1 - Sz
    
    aux1c = (Zt2/Zt1)*norm.cdf(d1)
    aux2c = strike*norm.cdf(d2)
    
    aux1p = strike*norm.cdf(-d2)
    aux2p = (Zt2/Zt1)*norm.cdf(-d1)
     
    call = Zt1 * (aux1c - aux2c)   
    put = Zt1 * (aux1p - aux2p)
    
    return call, put

In [14]:
tiempo, sim = sim_hw_many(gamma, sigma, theta, r0, num_sim, num_steps, seed )
# Extraemos las ultimas tasas(step) de cada simulacion
r= []
aux_Z=[]
for i in range(num_sim):
    aux = sim[i][-1]
    r.append(aux) # mis r
    
    aux1 = zero_hw(r[i], gamma, sigma, zrate, fwd, T_1, T_2)
    aux_Z.append(aux1) # mis precios del bono

In [15]:
#se simula los payoffs tanto para una call como para una put con la última tasa simulada (del step 264)
def sim_call_put(aux_Z, strike, num_sim):
    precios = []
    sim_call = []
    sim_put = []
    for i in range(num_sim):        
        precios.append(aux_Z[i])
        
        aux_call = max((precios[i]-strike),0)
        sim_call.append(aux_call)
        
        aux_put = max((strike - precios[i]),0)
        sim_put.append(aux_put)
        
    
    return sim_call, sim_put

In [16]:
sim_call, sim_put = sim_call_put(aux_Z, strike, num_sim)

In [17]:
#CÁLCULO VALOR PRESENTE DE LA OPCIÓN CALL Y PUT

In [18]:
#luego de esto, se obtienen el producto de los payoffs de la call y 
#la multiplicación de los factores de dcto simulados desde el día 1 hasta el día 263
#Finalmente, el valor presente de la opción call corresponde al promedio de estos productos 

products_call = []
for num1, num2 in zip(sim_call, df):
    products_call.append(num1 * num2)
    
#finalmente, se obtiene el valor presente del instrumento 
vp_call_sim = np.mean(products_call)

In [19]:
#Por otro lado, se obtienen el producto de los payoffs de la put y 
#la multiplicación de los factores de dcto simulados desde el día 1 hasta el día 263
#Finalmente, el valor presente de la opción put corresponde al promedio de estos productos 

products_put = []
for num1, num2 in zip(sim_put, df):
    products_put.append(num1 * num2)
#finalmente, se obtiene el valor presente del instrumento 
vp_put_sim = np.mean(products_put)

In [35]:
CallTeorico , PutTeorico = Bono_call_put(gamma, T_1, T_2, sigma, r0, zrate ,fwd ,t, strike )

print(f' Valor Teorico Call:{CallTeorico: .8} \n Valor Simulacion de Call:{vp_call_sim: .8}')
errorCall = abs(CallTeorico - vp_call_sim)
print(f' Error de la Call:{errorCall: .8}')

print(f' Valor Teorico Put:{PutTeorico: .8} \n Valor Simulacion de Put:{vp_put_sim: .8}')
errorPut = abs(PutTeorico-vp_put_sim)
print(f' Error de la Put:{errorPut: .8}')

 Valor Teorico Call: 0.0034467478 
 Valor Simulacion de Call: 0.0035198635
 Error de la Call: 7.3115671e-05
 Valor Teorico Put: 0.0040547718 
 Valor Simulacion de Put: 0.004025615
 Error de la Put: 2.915674e-05


In [36]:
# Tests

import hull_white as hw

call = hw.zcb_call_put(
    hw.CallPut.CALL,
    strike=strike,
    to=T_1,
    tb=T_2,
    r0=r0,
    zo=math.exp(-zrate(T_1) * T_1),
    zb=math.exp(-zrate(T_2) * T_2),
    gamma=gamma,
    sigma=sigma)

put = hw.zcb_call_put(
    hw.CallPut.PUT,
    strike=strike,
    to=T_1,
    tb=T_2,
    r0=r0,
    zo=math.exp(-zrate(T_1) * T_1),
    zb=math.exp(-zrate(T_2) * T_2),
    gamma=gamma,
    sigma=sigma)

print(call)
print(put)

0.0035283883401179272
0.00396076991674188


**AD:** Muy claro el procedimiento. Me hubiera gustado que probaran más strikes y plazos de vencimiento del bono cupón cero. El test del valor teórico no da perfecto.

In [22]:
############PREGUNTA 2: VALORIZANDO BONO 0 CUPÓN############

In [23]:
# Extraemos las ultimas tasas(step) de cada simulacion
last_sim = []
for i in range(num_sim):
    aux = sim[i][-1]
    last_sim.append(aux)

In [24]:
#se simula los payoffs del bono cero cupón con la última tasa simulada (del step 264)
def sim_bono(num_sim, last_sim, gamma, sigma, zrate, fwd, T_1, T_2):
    valorBono = []    
    for i in range(num_sim):
        r = last_sim[i]
        auxZ = zero_hw(r, gamma, sigma, zrate, fwd, T_1, T_2)
        valorBono.append(auxZ)
    return valorBono

In [25]:
valorBono_1Y = sim_bono(num_sim, last_sim, gamma, sigma, zrate, fwd, T_1, T_2)

In [26]:
#CÁLCULO VALOR PRESENTE DEL BONO CUPÓN 0 

In [27]:
#luego de esto, se obtienen el producto de los payoffs del bono y la multiplicación de los factores de dcto simulados desde el día 1 hasta el día 263
products = []

for num1, num2 in zip(valorBono_1Y, df):
    products.append(num1 * num2)
#finalmente, se obtiene el valor presente del instrumento 
vp_bono_sim = np.mean(products)

In [28]:
# En la celda anterior se quizo obtener el valor presente al igual que la opción call y put. 
#Sin embargo, luego nos dimos cuenta que el payoff del bono cupon 0 a vencimiento es igual a 1
#Por lo tanto, el valor presente de este instrumento se simplifica en cuanto al cálculo ya que solo corresponde 
#al promedio del producto de los factores de descuento diarios (del comienzo hasta el día 263)
vp_bono_sim = np.average(df)

In [29]:
len(df)

999

In [30]:
# finalmente el valor teórico de este instrumento corresponde al valor de la curva a un plazo de 1Y
vp_bono_teorico = curva['df'].iloc[15]

In [31]:
print(f'Valor Teorico del Bono:{vp_bono_teorico: .8} \nValor Simulado del Bono:{vp_bono_sim: .8}')
print(f'Error del bono:{abs(vp_bono_teorico-vp_bono_sim): .8}')

Valor Teorico del Bono: 0.99929787 
Valor Simulado del Bono: 0.99929008
Error del bono: 7.7906617e-06


**AD:** Igual que arriba. Ojalá hubieran testeado con más factores de descuento.